In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
from glob import glob
import os

directory = 'osman-parliament-edge-tts'
directory_text = 'osman-parliament-edge-tts-text'
mp3s = glob(os.path.join(directory, '*.mp3'))
jsons = glob(os.path.join(directory_text, '*.json'))
len(mp3s)

59582

In [3]:
# import IPython.display as ipd
# ipd.Audio('osman-news-edge-tts/46631.mp3')

In [4]:
# reject decimal points

In [5]:
import json

# for t in jsons:
#     with open(t) as fopen:
#         print(t, json.load(fopen))

In [6]:
from unidecode import unidecode
from malaya.text.normalization import digit, cardinal
import malaya
import re

_pad = 'pad'
_start = 'start'
_eos = 'eos'
_punctuation = "!'(),.:;? "
_special = '-'
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'
_numbers = '0123456789'

MALAYA_SPEECH_SYMBOLS = (
    [_pad, _start, _eos] + list(_special) + list(_punctuation) + list(_letters)
)

INITIAL_SYMBOLS = list(_letters) + list(_numbers)

/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/tensorflow_addons/utils/resource_loader.py:72: UserWarning: You are currently using TensorFlow 2.6.0 and trying to load a custom op (custom_ops/seq2seq/_beam_search_ops.so).
TensorFlow Addons has compiled its custom ops again

In [7]:
def tts_encode(string: str, add_eos: bool = True):
    r = [MALAYA_SPEECH_SYMBOLS.index(c) for c in string if c in MALAYA_SPEECH_SYMBOLS]
    if add_eos:
        r = r + [MALAYA_SPEECH_SYMBOLS.index('eos')]
    return r

In [8]:
def put_spacing_num(string):
    string = re.sub('[A-Za-z]+', lambda ele: ' ' + ele[0] + ' ', string)
    return re.sub(r'[ ]+', ' ', string).strip()

def convert_to_ascii(string):
    return unidecode(string)

def collapse_whitespace(string):
    return re.sub(_whitespace_re, ' ', string)

def put_spacing(string, chars = '()-'):
    for c in chars:
        string = string.replace(c, f' {c} ')
    return string

before = {';': ',', '_': '', '=': 'sama dengan', '*': 'asterisk',
          "'": '', '~': '', '`': ''}

after = {'/': 'garis miring'}

rejected = ['Coppa Italia', 'Pak Ramli memerlukan']

def replace_chars(string, chars):
    for k, v in chars.items():
        string = string.replace(k, f' {v} ')
    return string

patterns_num = [(r"\b\d+(?:[\.,']\d+)?\b\/\b\d+(?:[\.,']\d+)?\b", '/', 'garis miring'),
           (r"\b\d+(?:[\.,']\d+)?\b\-\b\d+(?:[\.,']\d+)?\b", '-', '')]

pattern_rm = r"RM \b\d+(?:[\.,']\d+)?\b (?:ribu|puluh|juta)"

replaces = {'dollar bilion': 'bilion dollar', 'dollar ribu': 'ribu dollar', 'dollar juta': 'juta dollar'}

def fix_pattern_num(string):
    for p in patterns_num:
        results = re.findall(p[0], string)
        for r in results:
            l_, r_ = r.split(p[1])
            string = string.replace(r, f'{digit(l_)} {p[2]} {digit(r_)}')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def fix_dash_num(string):
    results = re.findall(r"-\d+", string)
    for r in results:
        string = string.replace(r, f" {cardinal(r).replace('negatif', 'dash')} ")
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def fix_num_dash(string):
    results = re.findall(r"\d+-", string)
    for r in results:
        string = string.replace(r, f" {cardinal(r.replace('-', ''))} ")
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def cleaning(string, add_eos = False):
    sequence = []
    string = convert_to_ascii(string)
    string = replace_chars(string, before)
    string = fix_zeros(string)
    string = fix_dash_date(string)
    string = fix_date(string)
    string = fix_1900(string)
    string = fix_isbn(string)
    string = fix_pattern_num(string)
    string = fix_dash_num(string)
    string = fix_num_dash(string)
    string = fix_rm(string)
    string = put_spacing(string)
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = replace_chars(string, after)
    string = replace_chars(string, replaces)

    if string[-1] in '-,':
        string = string[:-1]
    if string[-2] in '-,!:;':
        string = string[:-2]
    if string[-1] != '.':
        string = string + '.'
    if string[0] not in INITIAL_SYMBOLS:
        string = string[1:]
    string = put_spacing_num(string)
    string = string.replace('dash', '')
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string, tts_encode(string, add_eos = add_eos)

def fix_dash_date(string):
    results = re.findall(r'\b\d+-\d+\/\d+\/\d+', string)
    for r in results:
        ori = r
        r = r.replace('-', ' dash ').replace('/', ' garis miring ')
        r = re.sub(r'[ ]+', ' ', r).strip()
        string = string.replace(ori, ' '.join([cardinal(i) for i in r.split()]))
    return string
        
def fix_rm(string):
    outer_results = {}
    t = string + ' .'
    compound = ['ribu', 'puluh', 'juta', 'bilion']
    pattern_rm = r"RM \b\d+(?:[\.,']\d+)?(?:[\.,']\d+)?\b (?:ribu|puluh|juta|bilion)"
    results = re.findall(pattern_rm, string)
    for r in results:
        splitted = r.split()
        if t[string.find(r) + len(r)] in '(/':
            s_ = splitted[-2].split('.')
            c = ' , '.join([cardinal(s__) for s__ in s_])
        else:
            c = cardinal(splitted[-2])
        if t[-2][-1] == '0' and '.' in splitted[-2]:
            c = f'{c} kosong'
        if t[string.find(r) + len(r)] in '(/':
            outer_results[r] = f'RM {c} {splitted[-1]}'.replace('perpuluhan', ',')
        else:
            outer_results[r] = f'{c} {splitted[-1]} RM'

    pattern_rm = r"RM \b\d+(?:[\.,']\d+)?(?:[\.,']\d+)?\b"
    results = re.findall(pattern_rm, string)
    for r in results:
        splitted = r.split()
        if t[string.find(r) + len(r)] in '(/':
            s_ = splitted[-1].split('.')
            c = ' , '.join([cardinal(s__) for s__ in s_])
        else:
            c = cardinal(splitted[-1])
        if splitted[-1][-1] == '0' and '.' in splitted[-1]:
            c = f'{c} kosong'
        if t[string.find(r) + len(r)] in '(/':
            outer_results[r] = f'RM {c}'.replace('perpuluhan', ',')
        else:
            outer_results[r] = f'{c} RM'
            
    for k, v in outer_results.items():
        string = string.replace(k, v)
    
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string

def fix_1900_year(string):
    results = re.findall(r'19\d\d-\d+', string)
    for r in results:
        splitted = r.split('-')
        string = string.replace(r, ' dash '.join(digit(s) for s in splitted))
    return string

def fix_1900(string):
    string = fix_1900_year(string)
    results = re.findall(r'19\d\d', string)
    for r in results:
        if r[-2:] != '00':
            if r[-2] == '0':
                c = 'kosong ' + cardinal(r[-1])
            else:
                c = cardinal(r[-2:])
            string = string.replace(r, 'sembilan belas ' + c)
        else:
            string = string.replace(r, 'sembilan belas ratus')
    return string

def fix_isbn(string):
    results = re.findall(r'[0-9\-]+', string)
    results_ = []
    for r in results:
        try:
            if string[:string.find(r) - 1].split()[-1].lower() == 'isbn':
                results_.append(r)
        except:
            pass
    for r in results_:
        splitted = r.split('-')
        string = string.replace(r, ' dash '.join([digit(s) for s in splitted]))
    return string

def fix_zeros(string):
    r = []
    for s in string.split():
        if len(re.findall(r'^(0+)\b', s)):
            s = digit(s)
        r.append(s)
    return ' '.join(r)

bulan = malaya.text.tatabahasa.bulan

def fix_date(string):
    short_date = re.findall(malaya.text.regex._short_date, string)
    if len(short_date):
        for r in short_date:
            if string[string.find(r) - 1] == '-':
                continue
            if '/' in r:
                splitted = r.split('/')
                if len(splitted) == 3:
                    if int(splitted[1]) > 12:
                        continue
                    day = cardinal(splitted[0])
                    month = bulan[int(splitted[1])].title()
                    year = cardinal(fix_1900(splitted[2]))
                else:
                    if int(splitted[0]) > 12:
                        continue
                    day = ''
                    month = bulan[int(splitted[0])].title()
                    year = cardinal(fix_1900(splitted[1]))
                string = string.replace(r, f'{day} {month} {year}')
            elif '.'in r:
                splitted = r.split('.')
                if len(splitted) == 3:
                    if len(splitted[-1]) == 2:
                        day = digit(splitted[0]) if splitted[0][0] == '0' else cardinal(splitted[0])
                        month = digit(splitted[1]) if splitted[1][0] == '0' else cardinal(splitted[1])
                        year = digit(splitted[2]) if splitted[2][0] == '0' else cardinal(splitted[2])
                        s = f'{day} {month} {year}'
                    else:
                        day = digit(splitted[0]) if splitted[0][0] == '0' else cardinal(splitted[0])
                        month = digit(splitted[1]) if splitted[1][0] == '0' else cardinal(splitted[1])
                        year = cardinal(fix_1900(splitted[2]))
                        s = f'{day} , {month} , {year}'
                    string = string.replace(r, s)
            elif '-' in r:
                splitted = r.split('-')
                if len(splitted) == 3:
                    if len(splitted[-1]) == 2:
                        day = digit(splitted[0]) if splitted[0][0] == '0' else cardinal(splitted[0])
                        month = digit(splitted[1]) if splitted[1][0] == '0' else cardinal(splitted[1])
                        year = digit(splitted[2]) if splitted[2][0] == '0' else cardinal(splitted[2])
                        s = f'{day} dash {month} dash {year}'
                    else:
                        day = digit(splitted[0]) if splitted[0][0] == '0' else cardinal(splitted[0])
                        month = digit(splitted[1]) if splitted[1][0] == '0' else cardinal(splitted[1])
                        year = cardinal(fix_1900(splitted[2]))
                        s = f'{day} dash {month} dash {year}'
                    string = string.replace(r, s)
                    
                
    date = re.findall(malaya.text.regex._date, string)
    if len(date):
        for r in date:
            if string[string.find(r) - 1] == '-':
                continue
            string = string.replace(r, ' '.join([cardinal(i) for i in r.split()]))
    return string

In [9]:
string = "Lapangan Terbang Egelsberg terletak pada kedudukan latitude N51 dua puluh tiga Julai seribu ' , longtituda E06 35.2500 ' pada ketinggian empat puluh dua perpuluhan sembilan lapan meter dari aras laut ."

In [10]:
r = cleaning(string)
r[0]

'Lapangan Terbang Egelsberg terletak pada kedudukan latitude N 51 dua puluh tiga Julai seribu , longtituda E 06 tiga puluh lima perpuluhan dua lima pada ketinggian empat puluh dua perpuluhan sembilan lapan meter dari aras laut .'

In [11]:
from tqdm import tqdm

acceptable = []
for t in tqdm(jsons):
    with open(t) as fopen:
        d = json.load(fopen)
        s = d['normalized']
        if len(re.findall(r"\b\d+\.\d+?\b", s)):
            continue
        r = cleaning(s)[0]
        if len(re.findall(r"\b\d+(?:[\.,']\d+)?\b", r)):
            continue
        acceptable.append((t, r))

100%|████████████████████████████████████████████████████████████████████████████████████████| 59582/59582 [00:17<00:00, 3452.35it/s]


In [12]:
acceptable[-10:]

[('osman-parliament-edge-tts-text/39041.json',
  'bahayanya penularan berita palsu , selain memberi penjelasan mengenai fungsi portal Sebenarnya . my .'),
 ('osman-parliament-edge-tts-text/50543.json',
  'Berdasarkan kepada semakan semula harga minyak kepada USDSS setong hasil Kerajaan Persekutuan dianggarkan berkurang sebanyak tiga belas bilion lapan ratus juta ringgit daripada anggaran asal .'),
 ('osman-parliament-edge-tts-text/39296.json',
  'Di samping itu , jumlah pendapatan pelancong asing turut meningkat daripada enam puluh lima bilion empat ratus juta ringgit satu sen pada tahun dua ribu tiga belas kepada tujuh puluh dua bilion ringgit pada tahun dua ribu empat belas .'),
 ('osman-parliament-edge-tts-text/15270.json',
  'Republik Rakyat China ( RRC ) merupakan rakan dagang terbesar Malaysia bagi sembilan tahun berturut - turut sejak dua ribu sembilan .'),
 ('osman-parliament-edge-tts-text/4211.json',
  'surat keputusan permohonan dihantar kepada pemohon dengan kerjasama Pejaba

In [13]:
len(acceptable), len(jsons)

(57130, 59582)

In [14]:
with open('postprocessing-edge-tts-parliament.json', 'w') as fopen:
    json.dump(acceptable, fopen)